In [39]:
!pip install pymupdf
!pip install faiss-cpu
!pip install langchain_nvidia_ai_endpoints
!pip install langchain_groq

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)


In [2]:
from langchain_community.document_loaders.pdf import PyMuPDFLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter

In [13]:
loader = PyMuPDFLoader('./pdfs/sd.pdf')
pages = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100)
chunks = splitter.split_documents(pages)
print(f"Split into {len(chunks)} chunks")

Split into 25 chunks


In [59]:
print(chunks[0])
chunk_texts = [chunk.page_content for chunk in chunks]
chunk_texts

page_content='Designing Instagram
Let's design a photo-sharing service like Instagram, where users can upload photos to share them with 
other users. Similar Services: Flickr, Picasa Difficulty Level: Medium
1. What is Instagram?
Instagram is a social networking service which enables its users to upload and share their photos and 
videos with other users. Instagram users can choose to share information either publicly or privately.' metadata={'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': './pdfs/sd.pdf', 'file_path': './pdfs/sd.pdf', 'total_pages': 6, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-02-15T04:25:03+00:00', 'trapped': '', 'modDate': 'D:20250215042503Z', 'creationDate': '', 'page': 0}


["Designing Instagram\nLet's design a photo-sharing service like Instagram, where users can upload photos to share them with \nother users. Similar Services: Flickr, Picasa Difficulty Level: Medium\n1. What is Instagram?\nInstagram is a social networking service which enables its users to upload and share their photos and \nvideos with other users. Instagram users can choose to share information either publicly or privately.",
 'Anything shared publicly can be seen by any other user, whereas privately shared content can only be \naccessed by a specified set of people. Instagram also enables its users to share through many other \nsocial networking platforms, such as Facebook, Twitter, Flickr, and Tumblr.\nFor the sake of this exercise, we plan to design a simpler version of Instagram, where a user can share \nphotos and can also follow other users. The ‘News Feed’ for each user will consist of top photos of all',
 'the people the user follows.\n2. Requirements and Goals of the System\n

In [ ]:
import faiss
from langchain_community.vectorstores import FAISS
from tqdm import tqdm

In [28]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

embeddings = NVIDIAEmbeddings(
  model="nvidia/llama-3.2-nv-embedqa-1b-v2", 
  api_key="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", 
  truncate="END", 
  )


In [29]:
batch_size = 50  # Matches NVIDIA's default max_batch_size
texts = [chunk.page_content for chunk in chunks]
metadatas = [chunk.metadata for chunk in chunks]

In [70]:
from tqdm.asyncio import tqdm_asyncio

progress_bar = tqdm_asyncio(total=len(texts), desc="Generating embeddings")

db = FAISS.from_texts(["init"], embeddings)

try:
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_metadatas = metadatas[i:i+batch_size]
        
        # Get embeddings using NVIDIA's async method
        batch_embeddings = await embeddings.aembed_documents(batch_texts)
        
        # Add to FAISS with metadata
        db.add_embeddings(
            text_embeddings=list(zip(batch_texts, batch_embeddings)),
            metadatas=batch_metadatas
        )
        
        # Update progress
        progress_bar.update(len(batch_texts))

except Exception as e:
    print(f"Error processing batch {i}: {str(e)}")
    raise
finally:
    progress_bar.close()

db.save_local(f"./database/sd")

Generating embeddings: 100%|██████████| 25/25 [00:02<00:00,  9.83it/s]


In [41]:
from langchain_groq import ChatGroq
import os

os.environ['GROQ_API_KEY'] = "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"
llm = ChatGroq(model='deepseek-r1-distill-llama-70b')

In [48]:
from langchain.prompts import PromptTemplate

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

prompt_template = """Below is an instruction that describes a task, paired with an context that provides further context.
Write a response that appropriately completes the request. If you don't know the answer to a question, please don't share false information. Just say that the question is out of scope.

### Instruction:
You are an electrical engineer and you will answer questions related to electrical engineering from the uploaded pdf context.

### Context:
{context}

### Question:
{question}

### Response:"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [52]:
from langchain_core.runnables import RunnablePassthrough
import re
from langchain_core.output_parsers import StrOutputParser

class RemoveChainOfThoughtOutputParser(StrOutputParser):
    def parse(self, text: str) -> str:
        # Remove everything between <think> and </think> (including the tags)
        cleaned_text = re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL)
        return cleaned_text.strip()

try:
   new_db = FAISS.load_local(f"./database/sd", embeddings, allow_dangerous_deserialization=True)
except FileNotFoundError as file_err:
    print(f"File not found: {str(file_err)}")
    raise

# initialize the RAG Chain
retriever = new_db.as_retriever()
rag_chain = (
  {"context": retriever | format_docs, "question": RunnablePassthrough()}
  | prompt
  | llm
  | RemoveChainOfThoughtOutputParser()
)

# try:
query = "What are we designing here?"
# This will automatically pass the query to both the retriever and the question slot.
answer = rag_chain.invoke(query)
print({'answer': answer})


{'answer': 'The system being designed is an Instagram-like platform focused on photo sharing. It is tailored for a social media application where users can upload, download, view, and search for photos, follow other users, and view a news feed. The design emphasizes handling a high volume of data efficiently, with considerations for separating read and write operations to manage server load and ensure quick retrieval of photos. The system is designed to scale, managing storage effectively and ensuring a smooth user experience.'}


In [69]:
import shutil

shutil.rmtree("./database/sd")

In [73]:
!pip install python-multipart

  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
Using cached python_multipart-0.0.20-py3-none-any.whl (24 kB)
